# App Store Analysis 
This analysis is for a company that only build free to download apps where main revenue drivers are in-app ads. \
This means that we need to analyse what types of apps are more likely to attract users 

The datasets and documentation can be found here here: 
* [Apple App Store Data](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps)
* [Google Play Store Data](https://www.kaggle.com/datasets/lava18/google-play-store-apps)

## Key Points
**Free to Play**
Since the company only uses free to download apps, the initial analysis cannot be taking into account the statistics from paid apps as this skews the data

**Data Points Across Both Apple and Google that can be used and are relevant**
* number of total downloads 
* ratings
* size of file
* content_rating
* genres
* language - we only want to be looking at apps that are targeted at English speaking clientele

**Data Point Relevant to Only Apple 
* screenshots shown for display 




## Initial Exploration and Data Cleaning 


In [1]:
def explore_data(dataset, start, end, rows_and_column = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') ##this will print a new line in between rows 
    
    if rows_and_column == True: 
        print('Number of rows: ', len(dataset))
        print('Number of columns' , len(dataset[0]))

The *explore_data* function has 4 params
* dataset - the dataset that has been opened that is to be explored 
* start - the beginning index *(or row)* of the slice of data being explored 
* end - the end index *(or row)* where the slice is being explored. Note that the end index is ***non inclusive***
* rows_and_columns - optional param to print information about nos of rows and columns of dataset

Below, we access the actual csv databases

In [2]:
opened_apple = open('/Users/wenmingsoh/desktop/datasets/AppleStore.csv', encoding = 'utf8')
opened_google = open('/Users/wenmingsoh/desktop/datasets/googleplaystore.csv',encoding = 'utf8')
from csv import reader 
read_apple = reader(opened_apple)
read_google = reader(opened_google)
apple_list =  list(read_apple)
google_list = list(read_google)




In [3]:
def check_row_length(dataset):
    error = []
    header = dataset[0]
    for row in dataset:
        if len(row) != len(header):
            print(row)
            print(dataset.index(row, 0, len(dataset)))
            error.append(row)
    if error == []:
        print('No More Errors')

## Cleaning the Google Data 

In [4]:
check_row_length(google_list)

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10473


In the code blocks above, we first created the **check_row_length** function to be see if there are rows that are a different length from the corresponding header of the dataset. \

Above we found that there was one wrong row in the google dataset. Below we will proceed to delete that row, and then run the function one more time 


In [5]:
del(google_list[10473])
check_row_length(google_list)

No More Errors


After deleting the erroneous row, and then running the **check_row_list** function one more time, we now find there are no more errors in *google_list*. Now we will do the same for *apple_list*

In [6]:
check_row_length(apple_list)

No More Errors


In [7]:
def check_for_duplicates(dataset, index):
    first_occurence = []
    duplicate = []
    for row in dataset[1:]:
        if row[index] in first_occurence:
            duplicate.append(row[index])
        else: 
            first_occurence.append(row[index])
    if duplicate == []:
        print("No Duplicates")
    else:
        print('Number of Duplicate Apps: ', len(duplicate))
        print('Examples of Duplicate Apps', duplicate[:15])

The **check_for_duplicates** function is used to check through an index of a dataset. \
The *index* param is used to give flexibility in how the function is used, as the index for the name of the app in each dataset might be different

For the *apple_list* dataset, the name is under **'track_name'**, index [1]
For the *google_list* dataset, the name is under **'App'**, index [0]

In [8]:
print('Duplicates for Google List')
print('\n')
check_for_duplicates(google_list, 0)

Duplicates for Google List


Number of Duplicate Apps:  1181
Examples of Duplicate Apps ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


Above, we see that there are multiple instances of duplicates in the google list. We need to investigate why. Examining the **Slack** app :

In [9]:
for row in google_list: 
    if row[0] == 'Slack':
        print(row)

['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51510', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']


We can see that the fourth index, index[3] has variances of number of user ratings. So the criterion for selecting the instance of each app that we use in our data analysis will be based on using the instance with the highest number of reviews (since that would be the most recent)

In [10]:
google_app_and_ratings = {}
app = ''
for row in google_list[1:]:
    app = row[0]
    if row[0] not in google_app_and_ratings:
        google_app_and_ratings[app] = int(row[3])
    else: 
        if int(row[3]) > google_app_and_ratings[app]:
            google_app_and_ratings[app] = int(row[3])
            
            

In [11]:
print("google_app_and_ratings length:",len(google_app_and_ratings))
print("Expected final outcome:", len(google_list[1:]) - 1181)

google_app_and_ratings length: 9659
Expected final outcome: 9659


The initial check_for_duplicates function which was run on the google_list showed that there were 1181 duplicates. So now we know that the dictionary google_app_and_ratings has the correct length

In [12]:
google_list_cleaned = []
already_added = []
for row in google_list[1:]:
    app_name = row[0]
    rating = row[3]
    if google_app_and_ratings[app_name] == int(row[3]) and app_name not in already_added:
        already_added.append(app_name)
        google_list_cleaned.append(row)

print(len(google_list_cleaned))
print(google_list_cleaned[0:2])

9659
[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']]


From here, we can deduce that the number of entries in the list is correct. Now we just need to add the header, and then do one more check.

In [13]:
google_list_cleaned.insert(0, google_list[0])
print(len(google_list_cleaned))
print(google_list_cleaned[0:3])

9660
[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']]


## Cleaning the Apple Data 

We now run the same initial proceses through the apple data, where we check the row length, and also check for duplicates. 

In [14]:
print('Duplicates for Apple List')
print('\n')
check_for_duplicates(apple_list, 1)

Duplicates for Apple List


Number of Duplicate Apps:  2
Examples of Duplicate Apps ['Mannequin Challenge', 'VR Roller Coaster']


In [15]:
check_row_length(apple_list)

No More Errors


From here, we can see that the apple list has the correct row lengths, so we don't need to fix that. However, there seem to be duplicate apps. So let's examine why

In [16]:
for row in apple_list: 
    if row[1] == "Mannequin Challenge" or row[1] == "VR Roller Coaster":
        print(row)

['1173990889', 'Mannequin Challenge', '109705216', 'USD', '0.0', '668', '87', '3.0', '3.0', '1.4', '9+', 'Games', '37', '4', '1', '1']
['952877179', 'VR Roller Coaster', '169523200', 'USD', '0.0', '107', '102', '3.5', '3.5', '2.0.0', '4+', 'Games', '37', '5', '1', '1']
['1178454060', 'Mannequin Challenge', '59572224', 'USD', '0.0', '105', '58', '4.0', '4.5', '1.0.1', '4+', 'Games', '38', '5', '1', '1']
['1089824278', 'VR Roller Coaster', '240964608', 'USD', '0.0', '67', '44', '3.5', '4.0', '0.81', '4+', 'Games', '38', '0', '1', '1']


The sixth index (*or index[5]*) is the user count across all versions. Using the same logic, we should take it that the one with the highest user count should be the most recent version, and the one that we would keep

In [17]:
mannequin_ratings = []
vr_roller_coaster_ratings = []

for row in apple_list:
    if row[1] == "Mannequin Challenge":
        mannequin_ratings.append(int(row[5]))
    elif row[1] == "VR Roller Coaster":
        vr_roller_coaster_ratings.append(int(row[5]))

mannequin_min = min(mannequin_ratings)
roller_coaster_min = min(vr_roller_coaster_ratings)

for row in apple_list: 
    if row[1] == "Mannequin Challenge" and int(row[5]) == mannequin_min:
        apple_list.remove(row)
    elif row[1] == "VR Roller Coaster" and int(row[5]) == roller_coaster_min:
        apple_list.remove(row)

The above block code is executed to remove the duplicates with the lower count of ratings. Now we do one more check to ascertain accuracy.

In [18]:
print('Duplicates for Apple List')
print('\n')
check_for_duplicates(apple_list, 1)

Duplicates for Apple List


No Duplicates


Now we know there are no more duplicates in apple_list. To reiterate, now we will be working with the two datasets:

* apple_list (still the same variable name as the initial list, but we've cleaned the data)
* google_list_cleaned (the completely cleaned google_list) 

For the sake of uniformity, we will now make a new list called apple_list_cleaned

In [19]:
apple_list_cleaned = []
for row in apple_list:
    apple_list_cleaned.append(row)
    
print(len(apple_list))
print(len(apple_list_cleaned))

7196
7196


As the two rows *apple_list* and *apple_list_cleaned* have the same number of rows, we know everything has copied over correctly. From here on we will be working with:

* apple_list_cleaned
* google_list_cleaned 

Now we will go on to remove all paid apps from these lists

## Removing Paid Apps from Apple Data

In [20]:
apple_list_free = []
google_list_free = []

def append_free(dataset, index, list_to_append):
    list_to_append.append(dataset[0])
    for row in dataset[1:]:
        if float(row[index]) == 0.0:
            row[index] = float(row[index])
            list_to_append.append(row)
            
append_free(apple_list_cleaned, 4, apple_list_free)

apple_count = 0
for row in apple_list_cleaned[1:]:
    if row[4] != float('0.0'):
        apple_count += 1

print('Number of Free Apple Apps expected to be removed: ', apple_count)
print('Difference between apple_list_cleaned and apple_list_free:', 
     len(apple_list_cleaned) - len(apple_list_free))

for row in apple_list_free[1:]:
    if row[4] != 0.0:
        print('Error')
    


Number of Free Apple Apps expected to be removed:  3141
Difference between apple_list_cleaned and apple_list_free: 3141


Having already done a check on all datatypes for price in the apple apps, and done a correlation check of number of expected items to be removed, we can ascertain that *apple_list_free* now has all the free apps accounted for.

## Removing Paid Apps from google_list_cleaned


In [21]:
for row in google_list_cleaned[1:]:
    if row[7] == '0':
        google_list_free.append(row)
        
google_count = 0 

for row in google_list_cleaned[1:]:
    if row[7] != '0':
        google_count += 1
        
print("Number of paid apps in google: ", google_count)
print("Check:", len(google_list_cleaned) - len(google_list_free) -1 )

        
        
        

Number of paid apps in google:  754
Check: 754


## Checking for Language of App Using Unicode


In [22]:
def check_string(string):
    counter = 0
    for character in string:
      if ord(character) > 127:
        counter += 1
    if counter >= 3:
        return False
    else:
        return True 
    

In [23]:
print(check_string('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(check_string('Instagram'))
print(check_string('Docs To Go™ Free Office Suite'))

False
True
True


The function above is built to run through all characters in a string - the filter is built to count the amount of unicode characters that are not in the native English language and will filter out any strings that have more than 3 unicode characters that have value greater than 127. This is to take into account certain peculiarities like emojis  

In [24]:
for row in google_list_free[:2000]:
    check_english = check_string(row[0])
    if check_english == False:
        print(row[0])
        

Truyện Vui Tý Quậy
Flame - درب عقلك يوميا
At home - rental · real estate · room finding application such as apartment · apartment
乐屋网: Buying a house, selling a house, renting a house
သိင်္ Astrology - Min Thein Kha BayDin


In [25]:
def remove_non_english(dataset, index, target_list):
    data_cleaned = f'{dataset=}'.split('=')[0]
    non_english_counter = 0
    for row in dataset:
        check_english = check_string(row[index])
        if check_english == False:
            non_english_counter += 1
        if check_english == True:
            target_list.append(row)
    print('-' * 10)
    print('Non English Apps Removed:', non_english_counter)
    print('Length of original dataset:', len(dataset))
    print('Length of dataset cleaned for non English apps', len(target_list))
    print('Difference in Entries:', len(dataset) - len(target_list))
    print('-' * 10)

google_list_free_english = []
apple_list_free_english = []

print('Cleaning Google Data')
remove_non_english(google_list_free, 0, google_list_free_english)
print('Cleaning Apple Data')
remove_non_english(apple_list_free, 1, apple_list_free_english)

            

Cleaning Google Data
----------
Non English Apps Removed: 57
Length of original dataset: 8905
Length of dataset cleaned for non English apps 8848
Difference in Entries: 57
----------
Cleaning Apple Data
----------
Non English Apps Removed: 853
Length of original dataset: 4055
Length of dataset cleaned for non English apps 3202
Difference in Entries: 853
----------


In [26]:
for row in google_list_free_english: 
    row[7] = float(row[7])

google_header = google_list_cleaned[0]
print(google_header)

google_list_free_english.insert(0, google_header)
print(google_list_free_english[:2])
print(apple_list_free_english[:2])
        

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', 0.0, 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']]
[['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'], ['284882215', 'Facebook', '389879808', 'USD', 0.0, '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']]


we want to ensure that both lists have headers, and also that all prices have been converted to floats. 

At this point, we have cleaned the data for both the apple and google related data. The data should now take into account apps that are :

* Only in English
* Only Free

As a final check, we run through the data in the cleaned lists, which will show errors if there are any apps that don't adhere to these conditions

In [27]:
def check_errors(dataset, name_index, price_index):
    non_english_count = 0
    non_free_count = 0
    for row in dataset:
        name = row[name_index]
        price = row[price_index]
        if check_string(name) == False:
            non_english_count += 1
        if price != 0.0:
            non_free_count += 1
    print("Non English Apps Found - ", non_english_count)
    print("Non Free Apps Found - ", non_free_count)

print("Checking Apple")
check_errors(apple_list_free_english[1:], 1, 4)
print("Checking Google")
check_errors(google_list_free_english[1:], 0, 7)
        
    

Checking Apple
Non English Apps Found -  0
Non Free Apps Found -  0
Checking Google
Non English Apps Found -  0
Non Free Apps Found -  0


At this point , are working with:
* apple_list_free_english
* google_list_free_english

From here, making an assumption that the company's strategy in building out new apps is :
* Build a minimal Android version of the app, and add it to Google Play.
* If the app has a good response from users, we develop it further.
* If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Since the end goal is to have a profitable app on both google and apple, research has to be done with the end goal in mind so that the buildout of the app has a better chance of being successful in both markets. 

**Relevant Categories from Apple:**
* rating_count-tot - User rating count for all versions
* user_rating - average user rating for all versions 
* cont_rating - content(age) rating 
* prime_rating = primary genre

**Relevant Categories from Google**
* Rating - Overall User Rating of app (as when scraped)
* Category - Main Category of app (like the main grouping on genres)
* Reviews - number of user reviews 
* Content Rating - Age group where app is targeted at
* *Genres - app can belong to multiple genres*




In [28]:
def freq_table(dataset, index):
    result_table = {}
    for row in dataset[1:]:
        if row[index] not in result_table:
            result_table[row[index]] = 1
        else:
            result_table[row[index]] += 1
    return result_table

def display_table(dataset, index):
    table = freq_table(dataset, index)
    display_table = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        display_table.append(key_val_as_tuple) 
        
    table_sorted = sorted(display_table, reverse = True)
    for entry in table_sorted:
        print(entry[1], " ", entry[0])


Two functions above 
**freq_table** - takes in a dataset and an index to create a dictionary that serves as a frequency table (number of occurences of the key in the key value pair occuring

**display_table** - takes the table created by **freq_table** function, extracts each key value pair into a tuple where the value as the first value into a tuple and each tuple is stored in the list display table. 
> This is required because of the need to sort the table later in descending order of occurences of frequency, which the initial dictionary format of the freq_table doesn't allow us to do because the keys are not the value that we are using to sort the entries by 

Once the sorting is done through the *sorted()* function, the result is stored in table_sorted which is a list of tuples with index[0] being the number of times entry has occured
Finally, the **display_table** function prints out index[1] then index index[0] of each list





In [29]:
display_table(apple_list_free_english, 11)

Games   1864
Entertainment   251
Photo & Video   160
Education   118
Social Networking   106
Shopping   83
Utilities   79
Sports   69
Music   66
Health & Fitness   65
Productivity   56
Lifestyle   50
News   43
Travel   40
Finance   35
Weather   28
Food & Drink   26
Reference   17
Business   17
Book   12
Navigation   6
Medical   6
Catalogs   4


In [30]:
display_table(google_list_free_english, 9)

Tools   747
Entertainment   538
Education   474
Business   407
Productivity   345
Lifestyle   343
Finance   328
Medical   313
Sports   306
Personalization   294
Communication   286
Action   274
Health & Fitness   273
Photography   261
News & Magazines   248
Social   236
Travel & Local   206
Shopping   199
Books & Reference   189
Simulation   181
Dating   165
Arcade   163
Video Players & Editors   157
Casual   156
Maps & Navigation   123
Food & Drink   110
Puzzle   100
Racing   88
Role Playing   83
Libraries & Demo   83
Auto & Vehicles   82
Strategy   81
House & Home   71
Weather   70
Events   63
Adventure   59
Comics   53
Beauty   53
Art & Design   53
Parenting   44
Card   40
Trivia   37
Casino   37
Educational;Education   35
Board   34
Educational   33
Education;Education   30
Word   23
Casual;Pretend Play   21
Music   18
Racing;Action & Adventure   15
Puzzle;Brain Games   15
Entertainment;Music & Video   15
Casual;Brain Games   12
Casual;Action & Adventure   12
Arcade;Action & Advent

The initial investigation shows that in terms of available apps, Apple has a large predisposition towards games and entertainment, but on google there is a bit more of an even distribution towards tools, education, business, productivity in addition to entertainment. 

As we are concerned with user engagement, we should be looking at correlating genres with metrics that would signify eyeballs/user engagement. These would be 

**Apple:**
* ratings_count_tot - used as a proxy for downloads since there are no stats for actual downloads
* user_rating - average user rating for all users (secondary metric) 

**Google:**
* Installs - number of installs
* Rating - Overall user rating of app

The first step would be to check the average number of engagements per app per genre for each platform

In [31]:
# #cleaning up google engagement values - at first engagement values are stored as strings, we need to change them
# to floats: 
for row in google_list_free_english[1:]:
    row[5] = row[5].replace('+', '')
    row[5] = row[5].replace(',', '')
    row[5] = float(row[5])

for row in google_list_free_english[1:]:
    if type(row[5]) != float:
        print(row)
    

In [32]:
#cleaning up apple engagement values = like above, all engagement numbers are stored as strings, we need to change
#them to floats 

for row in apple_list_free_english[1:]:
    row[5] = float(row[5])
    


In [33]:
def generating_avg_ratings(dataset, genre_index, interaction_index):
    total_ratings = {}##this will end up data in format key(genre): [total_interactions, app_count]
    for row in dataset[1:]:
        if row[genre_index] not in total_ratings: 
            total_ratings[row[genre_index]] = [row[interaction_index], 1]
        else:
            total_ratings[row[genre_index]][0] += row[interaction_index]
            total_ratings[row[genre_index]][1] += 1
    
    avr_rating_list = []
    
    for key, value in total_ratings.items():
        category = key
        data = value
        avr_rating_list.append([value[0] / value[1], key])
    
    return sorted(avr_rating_list, reverse = True)
    

The above function *generating_avg_ratings* has parameters for the dataset, genre, and the index that we will use metrics to track interaction. Using this function, we will extract the following data: 

* Apple Dataset: (dataset = apple_list_free_english, genre_index = 11(prime_genre), interaction_index = 5
* Google Dataset: (dataset = google_list_free_english, genre_index = 1(category), interaction_index = 5
* Google Dataset: (dataset = google_list_free_english, genre_index = 9(genres), interaction_index = 5



In [34]:
google_by_category = generating_avg_ratings(google_list_free_english, 1, 5)
for row in google_by_category[:10]:
    print(row[1], '|', row[0])

COMMUNICATION | 38590581.08741259
VIDEO_PLAYERS | 24727872.452830188
SOCIAL | 23253652.127118643
PHOTOGRAPHY | 17840110.40229885
PRODUCTIVITY | 16787331.344927534
GAME | 15544014.51048951
TRAVEL_AND_LOCAL | 13984077.710144928
ENTERTAINMENT | 11640705.88235294
TOOLS | 10830251.970588235
NEWS_AND_MAGAZINES | 9549178.467741935


In [35]:
google_by_genre = generating_avg_ratings(google_list_free_english, 9, 5)
for row in google_by_genre[:10]:
    print(row[1], '|', row[0])

Communication | 38590581.08741259
Adventure;Action & Adventure | 35333333.333333336
Video Players & Editors | 24947335.796178345
Social | 23253652.127118643
Arcade | 23028171.411042944
Casual | 19569221.602564104
Puzzle;Action & Adventure | 18366666.666666668
Photography | 17840110.40229885
Educational;Action & Adventure | 17016666.666666668
Productivity | 16787331.344927534


In [36]:
apple_by_prime_genre = generating_avg_ratings(apple_list_free_english, 11, 5)
for row in apple_by_prime_genre[:10]:
     print(row[1], '|', row[0])

Navigation | 86090.33333333333
Reference | 79350.4705882353
Social Networking | 71548.34905660378
Music | 57326.530303030304
Weather | 52279.892857142855
Book | 46384.916666666664
Food & Drink | 33333.92307692308
Finance | 32367.02857142857
Photo & Video | 28441.54375
Travel | 28243.8


## Working Through the Categories
While initial results show large engagement in the areas of communication and social networking across all datasets, we need also need to do a deeper analysis of the numbers to see if the numbers are being skewed by 'mega-apps' that are being run by competitors that we would have a tough time gaining market share from. 

As the company is also in the experimental phase or rolling out apps, we might not want to go into games which are known to be notoriously resource intensive to produce.

Navigation looks to be the highest score in terms of average engagement per app on IOS, and we should look through the data a bit more also


In [37]:
def find_outlier_values(dataset, engagement_index, app_index):
    total_engagement_count = 0
    list = []
    for row in dataset[1:]:
        total_engagement_count += row[engagement_index]
        if row[engagement_index] not in list:
            list.append([row[engagement_index], row[app_index]])
    
    sorted_list = sorted(list, reverse = True)
    truncated_sorted_list = sorted_list[0:20]
    for row in truncated_sorted_list:
        print(row[1], " : ", row[0], "|", round(row[0] * 100 / total_engagement_count, 2), '%')




In [38]:
print('Apple Data')
print('-' * 10)

find_outlier_values(apple_list_free_english, 5, 1)

Apple Data
----------
Facebook  :  2974676.0 | 3.72 %
Instagram  :  2161558.0 | 2.7 %
Clash of Clans  :  2130805.0 | 2.66 %
Temple Run  :  1724546.0 | 2.16 %
Pandora - Music & Radio  :  1126879.0 | 1.41 %
Pinterest  :  1061624.0 | 1.33 %
Bible  :  985920.0 | 1.23 %
Candy Crush Saga  :  961794.0 | 1.2 %
Spotify Music  :  878563.0 | 1.1 %
Angry Birds  :  824451.0 | 1.03 %
Subway Surfers  :  706110.0 | 0.88 %
Solitaire  :  679055.0 | 0.85 %
CSR Racing  :  677247.0 | 0.85 %
Crossy Road - Endless Arcade Hopper  :  669079.0 | 0.84 %
Injustice: Gods Among Us  :  612532.0 | 0.77 %
Hay Day  :  567344.0 | 0.71 %
PAC-MAN  :  508808.0 | 0.64 %
Calorie Counter & Diet Tracker by MyFitnessPal  :  507706.0 | 0.63 %
DragonVale  :  503230.0 | 0.63 %
The Weather Channel: Forecast, Radar & Alerts  :  495626.0 | 0.62 %


In [39]:
print('Google Data by Category')
print('-' * 10)
find_outlier_values(google_list_free_english, 5, 0)

Google Data by Category
----------
YouTube  :  1000000000.0 | 1.33 %
WhatsApp Messenger  :  1000000000.0 | 1.33 %
Subway Surfers  :  1000000000.0 | 1.33 %
Skype - free IM & video calls  :  1000000000.0 | 1.33 %
Messenger – Text and Video Chat for Free  :  1000000000.0 | 1.33 %
Maps - Navigate & Explore  :  1000000000.0 | 1.33 %
Instagram  :  1000000000.0 | 1.33 %
Hangouts  :  1000000000.0 | 1.33 %
Google+  :  1000000000.0 | 1.33 %
Google Street View  :  1000000000.0 | 1.33 %
Google Play Movies & TV  :  1000000000.0 | 1.33 %
Google Play Games  :  1000000000.0 | 1.33 %
Google Play Books  :  1000000000.0 | 1.33 %
Google Photos  :  1000000000.0 | 1.33 %
Google News  :  1000000000.0 | 1.33 %
Google Drive  :  1000000000.0 | 1.33 %
Google Chrome: Fast & Secure  :  1000000000.0 | 1.33 %
Google  :  1000000000.0 | 1.33 %
Gmail  :  1000000000.0 | 1.33 %
Facebook  :  1000000000.0 | 1.33 %


An initial look through shows that Facebook and Instagram score very high in terms of average engagement scores both on google and apple. Let's dive a bit deeper into the **social networking** category for apple


In [40]:
def check_low_max_total_engagement(dataset, category_index, category_name):
    social_engagement_array = []
    social_engagement_count = 0
    for row in apple_list_free_english[1:]:
        if row[11] == category_name:
            social_engagement_array.append(row[category_index])
            social_engagement_count += row[category_index]
    print("lowest score: ", min(social_engagement_array))
    print("max score: ", max(social_engagement_array))
    print("total social engagement count: ", social_engagement_count)

check_low_max_total_engagement(apple_list_free_english, 5, "Social Networking")
        

lowest score:  0.0
max score:  2974676.0
total social engagement count:  7584125.0


In [41]:
def check_high_engagement_apps(dataset, category, category_index, count_index, engagement_count, name_index):
    for row in dataset[1:]:
        if row[category_index] == category and row[count_index] > engagement_count:
            print(row[name_index], row[count_index])

check_high_engagement_apps(apple_list_free_english, "Social Networking",11, 5, 1000000.0, 1)

Facebook 2974676.0
Pinterest 1061624.0


In [42]:
print("Facebook and Pinterest take up ", (2974676.0 + 1061624.0) / 7584125.0 *100, "% of engagement counts")

Facebook and Pinterest take up  53.22037809239695 % of engagement counts


This shows us that two of the apps related to **Social Networking** on Apple already take up more than 50% of the total engagement counts - it is highly likely that these apps are skewing the data and that it will be very difficult to compete for market share 

In [43]:
check_low_max_total_engagement(apple_list_free_english, 5, "Navigation")
print("\n")
check_high_engagement_apps(apple_list_free_english, "Navigation", 11, 5, 100000.0, 1)
print("Waze and Google Maps take up ", (345046.0 + 154911.0) / 516542.0 *100, "% of engagement counts")

lowest score:  5.0
max score:  345046.0
total social engagement count:  516542.0


Waze - GPS Navigation, Maps & Real-time Traffic 345046.0
Google Maps - Navigation & Transit 154911.0
Waze and Google Maps take up  96.78922527112994 % of engagement counts


The skew for **Navigation** is even greater on Apple, where Waze and Google account for approx 97% of all engagement counts

In [44]:
check_low_max_total_engagement(apple_list_free_english, 5, "Reference")
check_high_engagement_apps(apple_list_free_english, "Reference", 11, 5, 250000.0, 1)
print("Bible takes up ", 985920.0 / 1348958.0 * 100, '% of engagement counts')

lowest score:  0.0
max score:  985920.0
total social engagement count:  1348958.0
Bible 985920.0
Bible takes up  73.08752385174334 % of engagement counts


The same pattern occurs in **Reference** - where Bible alone accounts for approx 73% of all engagement counts

In [45]:
check_low_max_total_engagement(apple_list_free_english, 5, "Music")
check_high_engagement_apps(apple_list_free_english, "Music", 11, 5, 500000.0, 1)
print("Pandora and Spotify Music take up ", (1126879.0 + 878563.0) / 3783551.0 *100, "% of engagement counts")

lowest score:  0.0
max score:  1126879.0
total social engagement count:  3783551.0
Pandora - Music & Radio 1126879.0
Spotify Music 878563.0
Pandora and Spotify Music take up  53.004228038686406 % of engagement counts


Same in **Music**

In [46]:
check_low_max_total_engagement(apple_list_free_english, 5, "Weather")
check_high_engagement_apps(apple_list_free_english, "Weather", 11, 5, 100000.0, 1)


lowest score:  0.0
max score:  495626.0
total social engagement count:  1463837.0
The Weather Channel: Forecast, Radar & Alerts 495626.0
The Weather Channel App for iPad – best local forecast, radar map, and storm tracking 208648.0
WeatherBug - Local Weather, Radar, Maps, Alerts 188583.0
MyRadar NOAA Weather Radar Forecast 150158.0
AccuWeather - Weather for Life 144214.0
Yahoo Weather 112603.0


The **weather** category shows a bit more promise, while the highest scoring app The Weather Channel: Forecast, Radar & Alerts takes up a large chunk of the total engagements, we see a fairly better distribution of engagement across apps as we move down the list

In [47]:
check_low_max_total_engagement(apple_list_free_english, 5, "Book")
check_high_engagement_apps(apple_list_free_english, "Book", 11, 5, 100000.0, 1)



lowest score:  0.0
max score:  252076.0
total social engagement count:  556619.0
Kindle – Read eBooks, Magazines & Textbooks 252076.0
Audible – audio books, original series & podcasts 105274.0


Here we see two things: 
* **Kindle** and **Audible** account for a large chunk of the **Book** Engagements
* Apple categorizes **Book** more as book management apps, which means that it may be storing actual books under **Reference**

Let's further examine **Weather** and **Reference** and see what the engagement counts are like if we remove the counts for both category's top apps

In [48]:
def remove_top_app(dataset, category_index, engagement_index, category_name):
    total_engagement = 0
    highest_engagement = 0
    for row in dataset[1:]:
        if row[category_index] == category_name:
            total_engagement += row[engagement_index]
            if row[engagement_index] > highest_engagement:
                highest_engagement = row[engagement_index]
    print("Total Engagement:", total_engagement)
    print("Highest Engagement: ", highest_engagement)
    print("Total - Highest Engagement :", total_engagement - highest_engagement)
    
print('-Weather-')
remove_top_app(apple_list_free_english, 11, 5, "Weather")
print('-Reference-')
remove_top_app(apple_list_free_english, 11, 5, "Reference")


-Weather-
Total Engagement: 1463837.0
Highest Engagement:  495626.0
Total - Highest Engagement : 968211.0
-Reference-
Total Engagement: 1348958.0
Highest Engagement:  985920.0
Total - Highest Engagement : 363038.0


Based on what we see above - it looks like **weather** might be worth investgating. But we can also keep **reference** in our back pocket

In [49]:
for row in google_list_free_english[1:]:
    if row[1] == "WEATHER":
        print(row[0])


The Weather Channel: Rain Forecast & Storm Alerts
Weather forecast
AccuWeather: Daily Forecast & Live Weather Reports
Live Weather Pro
Weather by WeatherBug: Forecast, Radar & Alerts
weather - weather forecast
MyRadar NOAA Weather Radar
SMHI Weather
Free live weather on screen
Weather Radar Widget
Weather –Simple weather forecast
Weather Crave
Klara weather
Yahoo Weather
Real time Weather Forecast
METEO FRANCE
APE Weather ( Live Forecast)
Live Weather & Daily Local Weather Forecast
Weather
Rainfall radar - weather
Yahoo! Weather for SH Forecast for understanding the approach of rain clouds Free
The Weather Network
Klart.se - Sweden's best weather
GO Weather - Widget, Theme, Wallpaper, Efficient
Info BMKG
Weather From DMI/YR
wetter.com - Weather and Radar
Storm Radar: Tornado Tracker & Hurricane Alerts
Yandex.Weather
Local Weather Forecast & Visual Widget
Wetter by t-online.de
HTC Weather
AEMET's time
New 2018 Weather App & Widget
Météociel
Climatempo Lite - 15 day weather forecast
Fore

The main issue with proposing developing an app for the **weather** category, even though there seems to be a good spread of interactions is that from a business perspective it would be difficult to differentiate the product to get users to download and interact over the current apps that are available.

Knowing that in a lot of categories, the top 2 apps seem to skew the statistics for engagement per app (on a category basis), let's reexamine the data to see what happens when we remove the top 2 apps from each category. We'll work with the **Apple** dataset first, since engagement numbers are more granular

In [116]:
def delete_top_2(dataset, engagement_index, category_index):
    category_dict = {}
    for row in dataset[1:]:
        if row[category_index] not in category_dict:
            category_dict[row[category_index]] = [row[engagement_index]]
        else:
            category_dict[row[category_index]].append(row[engagement_index])
           
    for key in category_dict:
        category_dict[key] = sorted(category_dict[key], reverse = True)
        category_dict[key] = category_dict[key][2:]
   
    sorted_list = []
    counter = 0.0
    summer = 0.0
    for key, value in category_dict.items():
        for entry in value:
            counter += 1
            summer += entry
        sorted_list.append([summer/counter, key])
        counter = 0.0
        summer = 0.0
    
    sorted_list = sorted(sorted_list, reverse = True)
    for row in sorted_list[:10]:
        print(row[1], '|', row[0])
    
            
            
        

         

In [117]:
delete_top_2(apple_list_free_english, 5, 11)

Social Networking | 34113.70192307692
Weather | 29213.96153846154
Music | 27782.953125
Games | 20864.89688506982
Finance | 20201.090909090908
Book | 19926.9
Shopping | 19507.79012345679
Sports | 16506.68656716418
Productivity | 16306.74074074074
Photo & Video | 13070.784810126583


## Apple Statistics
|With all Apps Counted|Avr Eng Per App Per Category|With Top 2 Apps Rem|Avr Eng Per App Per Category|
|   :---:    |            :---:     |  :---:                       |:---:|
|Navigation | 86090.33333333333                |Social Networking | 34113.70192307692|
|Reference | 79350.4705882353                  |Weather | 29213.96153846154|
|Social Networking | 71548.34905660378         |Music | 27782.953125|
|Music | 57326.530303030304|Games | 20864.89688506982|
|Weather | 52279.892857142855|Finance | 20201.090909090908|
|Book | 46384.916666666664|Book | 19926.9|
|Food & Drink | 33333.92307692308|Shopping | 19507.79012345679|
|Finance | 32367.02857142857| Sports | 16506.68656716418|
|Photo & Video | 28441.54375| Productivity | 16306.74074074074|
|Travel | 28243.8| Photo & Video | 13070.784810126583|

Eliminating the top 2 Apps for each category and recalcuating the average engagements per app per customer, interestingly we find out a few things 

* The book category remains at rank 6, however from a pure quantum perspective, the difference between the book category and the top app category is approx 40000 engagements per app without discounting the top 2 apps, but when we discount the top 2 apps the difference between the book category and the top app category is approx 14000 engagements
* Finance moves up the ranks to rank 5 in Avr Eng Per App Per Category when we eliminate the top 2 apps in each category 

Given that we see the relative strength of the **Book** category after running this exercise, we should investigate the book category more, and also look into the google data for more information

In [128]:
book_list = []
for row in apple_list_free_english[1:]:
    if row[11] == "Book":
       book_list.append(row[1])
for row in book_list:
    print(row)

Kindle – Read eBooks, Magazines & Textbooks
Audible – audio books, original series & podcasts
Color Therapy Adult Coloring Book for Adults
OverDrive – Library eBooks and Audiobooks
HOOKED - Chat Stories
BookShout: Read eBooks & Track Your Reading Goals
Dr. Seuss Treasury — 50 best kids books
Green Riding Hood
Weirdwood Manor
MangaZERO - comic reader
ikouhoushi
MangaTiara - love comic reader


In [129]:
book_list = []
for row in apple_list_free_english[1:]:
    if row[11] == "Reference":
       book_list.append(row[1])
for row in book_list:
    print(row)

Bible
Dictionary.com Dictionary & Thesaurus
Dictionary.com Dictionary & Thesaurus for iPad
Google Translate
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition
Merriam-Webster Dictionary
Night Sky
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE)
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools
GUNS MODS for Minecraft PC Edition - Mods Tools
Guides for Pokémon GO - Pokemon GO News and Cheats
WWDC
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free
VPN Express
Real Bike Traffic Rider Virtual Reality Glasses
Jishokun-Japanese English Dictionary & Translator


Looking through the categories for **Books** and **Reference**, we find that there are some peculiarities in categorization - technically the Bible and Dictionaries could be categorized as books, but we see them appear in reference. 

We see in the **Books** category that there might be an opportunity to create focused, small apps that feature either a singular book, or a small collection of books that is part of popular culture. If the app covers a smaller breadth of subject matter, perhaps we could build additional features into the app to drive engagement such as 

* Quizzes
* Graphical Interaction
* Social Elements in Reading (read with your friends)

Let's now look through the google data to take another look. 


In [132]:
book_list = []
for row in google_list_free_english[1:]:
    if row[9] == "Books & Reference":
       book_list.append(row[0])
for row in book_list:
    print(row)

E-Book Read - Read Book for free
Download free book with green book
Wikipedia
Cool Reader
Free Panda Radio Music
Book store
FBReader: Favorite Book Reader
English Grammar Complete Handbook
Free Books - Spirit Fanfiction and Stories
Google Play Books
AlReader -any text book reader
Offline English Dictionary
Offline: English to Tagalog Dictionary
FamilySearch Tree
Cloud of Books
Recipes of Prophetic Medicine for free
ReadEra – free ebook reader
Anonymous caller detection
Ebook Reader
Litnet - E-books
Read books online
English to Urdu Dictionary
eBoox: book reader fb2 epub zip
English Persian Dictionary
Flybook
All Maths Formulas
Ancestry
HTC Help
English translation from Bengali
Pdf Book Download - Read Pdf Book
Free Book Reader
eBoox new: Reader for fb2 epub zip books
Only 30 days in English, the guideline is guaranteed
Moon+ Reader
SH-02J Owner's Manual (Android 8.0)
English-Myanmar Dictionary
Golden Dictionary (EN-AR)
All Language Translator Free
Azpen eReader
URBANO V 02 instruction 

Here we see the topics are varied, however there are multiple examples of books that cover the subject matter of 

* The Quran
* Computer Programming

It would seem that there is opportunity in the market for having multiple books that cover the same subject matter, but perhaps have differentiation in in app features to drive greater engagement


### Conclusion

Having taken all the data points that have been mined into consideration and adding a certain amount of market understanding of the app market in general, we recommend that the **Book** Category gives the highest chance of ROI if the company is looking to experiment with developing apps that are :

* Free
* Targeted to English Speaking Customers
* Not in a category that is highly dominated by competitors with large market share and a large amount of resources 
* Not in a category where the development process can be highly resource intensive 

The recommendation is to focus on building book apps that have cover a concentrated subject matter - book apps that have mass appeal (such as books that would be relevant to popular culture) would provide such opportunities, with opportunities for product differentiation through features discussed earlier. 
